In [1]:
#import packages:

import os
os.environ['NUMBAPRO_NVVM'] = '/usr/local/cuda/nvvm/lib64/libnvvm.so'
os.environ['NUMBAPRO_LIBDEVICE'] = '/usr/local/cuda/nvvm/libdevice/'

from google.colab import drive

try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

if IN_COLAB:
  # Check if drive is mounted by looking for the mount point in the file system.
  # This is a more robust approach than relying on potentially internal variables.
  import os
  if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')
  else:
    print("Google Drive is already mounted.")
else:
  print("Not running in Google Colab. Drive mounting skipped.")


import os
from google.colab import drive
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from tqdm import tqdm


Google Drive is already mounted.


In [ ]:

#if using colabs - will need to first mount your drive

#change these for different users

# File paths (update if needed)
data_filepath = '/content/drive/MyDrive/Colab_Notebooks/Data/UNSW_NB15_testing-set.parquet'

# Load data
test_set = pd.read_parquet(test_set_filepath)
train_set = pd.read_parquet(training_set_filepath)

print("Data loaded")



Data loaded


In [ ]:
# Preprocessing function (modified for pandas)
def preprocess_data(data_set):
  if 'attack_cat' in data_set.columns.tolist():
    data_set = data_set.drop('attack_cat', axis=1)

  if 'proto' in data_set.columns.tolist():
    category_percentages = data_set['proto'].value_counts(normalize=True) * 100

    top_6_categories = category_percentages.head(6).index.tolist()

    data_set['proto_grouped'] = data_set['proto'].apply(lambda x: x if x in top_6_categories else 'other')

    data_set = pd.get_dummies(data_set, columns=['proto_grouped'], prefix='proto_grouped')

    data_set = data_set.drop('proto', axis=1)

  if 'proto_grouped' in data_set.columns.tolist():
      data_set = data_set.drop(['proto_grouped'], axis=1)

  categorical_cols = data_set.select_dtypes(include=['category']).columns.tolist()
  data_set = pd.get_dummies(data_set, columns=categorical_cols, prefix_sep='_')

  binary_cols = data_set.select_dtypes(include=['bool']).columns
  data_set[binary_cols] = data_set[binary_cols].astype(int)

  print(f"Data set preprocessed, columns = {data_set.columns.tolist()}")
  return data_set

#note to self - train set and test set will have different columns after one-hot encoding, due to different categorical values
train_set = preprocess_data(train_set)
test_set = preprocess_data(test_set)

#add columns called 'state_URN', 'state_no' to test_set dataframe and fill with zeros

# Get the difference in columns between the two DataFrames
train_set_cols = set(train_set.columns)
test_set_cols = set(test_set.columns)

missing_in_test = list(train_set_cols - test_set_cols)
missing_in_train = list(test_set_cols - train_set_cols)

# Add missing columns to test_set and train_set, filled with 0s
for col in missing_in_test:
    test_set[col] = 0

for col in missing_in_train:
    train_set[col] = 0

#ensure same order
test_set = test_set[train_set.columns]

# Verify that both DataFrames now have the same columns
print(f"Train set columns: {train_set.columns.tolist()}")
print(f"Test set columns: {test_set.columns.tolist()}")

assert set(train_set.columns) == set(test_set.columns), "DataFrames do not have the same columns"




Data set preprocessed, columns = ['dur', 'spkts', 'dpkts', 'sbytes', 'dbytes', 'rate', 'sload', 'dload', 'sloss', 'dloss', 'sinpkt', 'dinpkt', 'sjit', 'djit', 'swin', 'stcpb', 'dtcpb', 'dwin', 'tcprtt', 'synack', 'ackdat', 'smean', 'dmean', 'trans_depth', 'response_body_len', 'ct_src_dport_ltm', 'ct_dst_sport_ltm', 'is_ftp_login', 'ct_ftp_cmd', 'ct_flw_http_mthd', 'is_sm_ips_ports', 'label', 'proto_grouped_arp', 'proto_grouped_ospf', 'proto_grouped_other', 'proto_grouped_sctp', 'proto_grouped_tcp', 'proto_grouped_udp', 'proto_grouped_unas', 'service_-', 'service_dhcp', 'service_dns', 'service_ftp', 'service_ftp-data', 'service_http', 'service_irc', 'service_pop3', 'service_radius', 'service_smtp', 'service_snmp', 'service_ssh', 'service_ssl', 'state_CON', 'state_ECO', 'state_FIN', 'state_INT', 'state_PAR', 'state_REQ', 'state_RST', 'state_URN', 'state_no', 'state_CLO', 'state_ACC']
Data set preprocessed, columns = ['dur', 'spkts', 'dpkts', 'sbytes', 'dbytes', 'rate', 'sload', 'dload', 

With a modest class imbalance, these ROC AUC results show that the data is very linearly separable. This is beacuse A. Logistic Regression performs very well, and B. there isn't a large improvement when switching to a Decision Tree model (and an even smaller improvement moving to Random Forest).

NOTE - Therefore, there is little justification for using a more complex, less interpetable model on this dataset. I'll choose another: this project is effectively cancelled.

Average Validation ROC AUC from nested cross-validation (Logistic Regression): 0.9619402618207303

Average Validation ROC AUC from nested cross-validation (Decision Tree): 0.9856269426697901

Average Validation ROC AUC from nested cross-validation (Random Forest): 0.9889679397775413
